In [18]:
from flair.data import TaggedCorpus
from flair.data_fetcher import NLPTaskDataFetcher
from flair.embeddings import TokenEmbeddings, WordEmbeddings, StackedEmbeddings, PooledFlairEmbeddings
from typing import List
from flair.visual.training_curves import Plotter

In [19]:
from flair.data import Sentence
from flair.embeddings import TokenEmbeddings, WordEmbeddings, StackedEmbeddings, PooledFlairEmbeddings, FlairEmbeddings

# initialize embeddings. This takes time to load.
embedding_types: List[TokenEmbeddings] = [
    # GloVe embeddings for arabic
#     WordEmbeddings('ar'),
    FlairEmbeddings('/home/jupyter/language_model/wiki-forward/best-lm.pt'), 
    FlairEmbeddings('/home/jupyter/language_model/wiki-backward/best-lm.pt')
]

embeddings: StackedEmbeddings = StackedEmbeddings(embeddings=embedding_types)

In [20]:

# 1. get the corpus
columns = {0: 'text', 1: 'ner'}
data_folder = '/home/jupyter/data_ner/'
corpus: TaggedCorpus = NLPTaskDataFetcher.load_column_corpus(data_folder, columns, 
  train_file='train.txt', 
  dev_file='dev.txt')

# 2. what tag do we want to predict?
tag_type = 'ner'

# 3. make the tag dictionary from the corpus
tag_dictionary = corpus.make_tag_dictionary(tag_type=tag_type)


# initialize sequence tagger
from flair.models import SequenceTagger

# have a relatively small hidden_size
tagger: SequenceTagger = SequenceTagger(hidden_size=64,
                                        dropout = 0.2,
                                        rnn_layers = 2,
                                        embeddings=embeddings,
                                        tag_dictionary=tag_dictionary,
                                        tag_type=tag_type,
                                        use_crf=True)# initialize trainer
# 6. Initialize trainer
from flair.trainers import ModelTrainer
from flair.training_utils import EvaluationMetric

from flair.optim import AdamW
# optimizer = optimizer(momentum=0.9, dampening=0.9)
trainer: ModelTrainer = ModelTrainer(tagger, corpus, optimizer=AdamW)


2019-04-15 16:39:41,700 Reading data from /home/jupyter/data_ner
2019-04-15 16:39:41,701 Train: /home/jupyter/data_ner/train.txt
2019-04-15 16:39:41,703 Dev: /home/jupyter/data_ner/dev.txt
2019-04-15 16:39:41,704 Test: None


In [21]:
import os
# 7. start training
result_folder = '/home/jupyter/result_char_only/'
trainer.train(result_folder,
              EvaluationMetric.MICRO_F1_SCORE,
              learning_rate=0.1,
              mini_batch_size=32,
              weight_decay = 0.1,
              max_epochs=200,
              checkpoint=False)


2019-04-15 16:39:53,050 ----------------------------------------------------------------------------------------------------
2019-04-15 16:39:53,052 Evaluation method: MICRO_F1_SCORE
2019-04-15 16:39:53,053 ----------------------------------------------------------------------------------------------------
2019-04-15 16:39:53,481 epoch 1 - iter 0/38 - loss 61.07507324
2019-04-15 16:39:55,211 epoch 1 - iter 3/38 - loss 88.63649559
2019-04-15 16:39:56,560 epoch 1 - iter 6/38 - loss 60.63908168
2019-04-15 16:39:57,967 epoch 1 - iter 9/38 - loss 47.55303011
2019-04-15 16:39:59,860 epoch 1 - iter 12/38 - loss 42.99303194
2019-04-15 16:40:01,305 epoch 1 - iter 15/38 - loss 38.34075266
2019-04-15 16:40:02,937 epoch 1 - iter 18/38 - loss 35.11664305
2019-04-15 16:40:04,502 epoch 1 - iter 21/38 - loss 32.64207628
2019-04-15 16:40:06,250 epoch 1 - iter 24/38 - loss 30.87922886
2019-04-15 16:40:08,534 epoch 1 - iter 27/38 - loss 29.40119161
2019-04-15 16:40:10,306 epoch 1 - iter 30/38 - loss 28.1

KeyboardInterrupt: 

In [14]:
from flair.visual.training_curves import Plotter
plotter = Plotter()
plotter.plot_training_curves(os.path.join(result_folder, 'loss.tsv'))
# plotter.plot_weights(os.path.join(result_folder, 'weights.txt'))

In [15]:
from predict_from_file_to_csv import *
model_folder = result_folder 
from_model_to_solution(model_folder)

2019-04-15 15:38:17,316 loading file /home/jupyter/result_word_and_char/final-model.pt
load model okay
dev.txt,20214,20214,0.98,B-MIS,"الإلكترونات"

test.txt,18884,18884,0.96,I-ORG,"مانشستر"

train.txt,37362,37362,0.97,B-LOC,"بردى"

